Converting features into ones we can actually use, and moving the target value to the last column.

In [41]:
import numpy as np
import pandas as pd

In [42]:
df = pd.read_csv('data_with_target.csv')


In [43]:
df.head()

,Unnamed: 0,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,...,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape
0,0,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,511,...,A,40.036944,-121.005833,5.0,USFS,CA,63,63.0,Plumas,b'\x00\x01\xad\x10\x00\x00\xe8d\xc2\x92_@^\xc0...
1,1,2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,...,A,38.933056,-120.404444,5.0,USFS,CA,61,61.0,Placer,b'\x00\x01\xad\x10\x00\x00T\xb6\xeej\xe2\x19^\...
2,2,3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,...,A,38.984167,-120.735556,13.0,STATE OR PRIVATE,CA,17,17.0,El Dorado,b'\x00\x01\xad\x10\x00\x00\xd0\xa5\xa0W\x13/^\...
3,3,4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,...,A,38.559167,-119.913333,5.0,USFS,CA,3,3.0,Alpine,b'\x00\x01\xad\x10\x00\x00\x94\xac\xa3\rt\xfa]...
4,4,5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,...,A,38.559167,-119.933056,5.0,USFS,CA,3,3.0,Alpine,b'\x00\x01\xad\x10\x00\x00@\xe3\xaa.\xb7\xfb]\...


In [ ]:
 #convert the 4 columns in time_cols into datetime objects
# then calculate the timedelta 
#Slightly changed from original version in calc_cont_time to force .1f formatting
def make_time(row):
    # pad leading 0s to front of time so 438 = 0438, etc
    disc_time = '{0:.1f}'.format(row['DISCOVERY_TIME']).zfill(4)
    disc_time = datetime.strptime(disc_time, '%H%M.%S').time()
    cont_time = '{0:.1f}'.format(row['CONT_TIME']).zfill(4)
    cont_time = datetime.strptime(cont_time, '%H%M.%S').time()
    # dates are formatted as Julian, convert to datetime
    disc_date = pd.to_datetime(row['DISCOVERY_DATE'] - epoch, unit='D')
    cont_date = pd.to_datetime(row['CONT_DATE'] - epoch, unit='D')
    # combine the time and the date
    disc = datetime.combine(disc_date, disc_time)
    cont = datetime.combine(cont_date, cont_time)
    delta = cont-disc 
    return delta

df['TIME_TO_CONT'] = df.apply(lambda row: make_time(row), axis=1)

In [56]:
df.head()

,Unnamed: 0,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,...,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape,TIME_TO_CONT
0,0,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,511,...,40.036944,-121.005833,5.0,USFS,CA,63,63.0,Plumas,b'\x00\x01\xad\x10\x00\x00\xe8d\xc2\x92_@^\xc0...,0 days 04:30:00
1,1,2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,...,38.933056,-120.404444,5.0,USFS,CA,61,61.0,Placer,b'\x00\x01\xad\x10\x00\x00T\xb6\xeej\xe2\x19^\...,0 days 06:45:00
2,2,3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,...,38.984167,-120.735556,13.0,STATE OR PRIVATE,CA,17,17.0,El Dorado,b'\x00\x01\xad\x10\x00\x00\xd0\xa5\xa0W\x13/^\...,0 days 01:03:00
3,3,4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,...,38.559167,-119.913333,5.0,USFS,CA,3,3.0,Alpine,b'\x00\x01\xad\x10\x00\x00\x94\xac\xa3\rt\xfa]...,4 days 22:00:00
4,4,5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,...,38.559167,-119.933056,5.0,USFS,CA,3,3.0,Alpine,b'\x00\x01\xad\x10\x00\x00@\xe3\xaa.\xb7\xfb]\...,4 days 20:00:00


In [57]:
df['TIME_TO_CONT'].describe()

count                     892007
mean      1 days 06:47:50.565881
std      13 days 19:32:53.401533
min            -1 days +03:36:00
25%              0 days 00:30:00
50%              0 days 01:29:00
75%              0 days 05:00:00
max           3653 days 01:30:00
Name: TIME_TO_CONT, dtype: object

In [ ]:
df.to_csv("data_with_cont_time.csv")